# 🎤 VoiceScribe AI - Complete Implementation

**Intelligent Audio Analysis Tool** - Transforms audio recordings into actionable insights

This notebook contains the complete implementation that you can run step by step.

---

## 🔹 Features:

- 🎤 Automatic Transcription with Whisper
- 🧠 Smart Summarization with BART
- 📌 Key Points Extraction
- 🔍 Searchable Transcript
- 💬 Chat with Audio (RAG Q&A)
- 📂 Gradio UI Interface

---

## 📖 How to Use This Notebook

Run each cell sequentially from top to bottom:
1. Install dependencies (Step 1)
2. Import libraries (Step 2)
3. Define data structures (Step 3)
4. Create the VoiceScribeAI class (Step 4)
5. Build Gradio interface (Step 5)
6. Launch the application (Step 6)

## Step 1: Install Dependencies

First, let's install all required packages. This may take 5-10 minutes.

**What we're installing:**
- `gradio` - Web UI framework
- `torch` - Deep learning framework
- `transformers` - AI models from Hugging Face
- `langchain` - RAG framework for Q&A (with compatible versions)
- `faiss-cpu` - Vector database for similarity search
- `sentence-transformers` - Text embeddings
- `librosa`, `soundfile` - Audio processing

**Note:** If you see dependency conflicts, re-run this cell to fix them.

## Step 0: Fix Dependency Conflicts (Run This First If Needed)

**Only run this cell if you're seeing dependency conflicts!** This will clean up conflicting packages.

After running this, proceed to Step 1.

In [ ]:
# Clean up conflicting langchain packages
print("🧹 Cleaning up conflicting packages...")

!pip uninstall -y langchain-core langchain langchain-community langchain-huggingface langchain-text-splitters 2>/dev/null

print("✅ Cleanup complete! Now run Step 1 to install correct versions.")

In [ ]:
# Install core dependencies
!pip install -q gradio torch transformers accelerate

# Install audio processing libraries first
!pip install -q librosa soundfile pydub

# Install RAG and vector store with carefully managed versions
# Install langchain-core first with the right version
!pip install -q "langchain-core>=0.3.72,<1.0.0"

# Install other langchain packages that depend on langchain-core < 1.0
!pip install -q "langchain>=0.3.0,<0.4.0"
!pip install -q "langchain-community>=0.3.0,<0.4.0"
!pip install -q "langchain-text-splitters>=0.3.0,<0.4.0"

# Install langchain-huggingface with compatible version (< 1.0.0)
!pip install -q "langchain-huggingface>=0.0.1,<1.0.0"

# Install FAISS and sentence transformers
!pip install -q faiss-cpu sentence-transformers

print("✅ All dependencies installed!")
print("✅ Dependency conflicts resolved!")

## Step 2: Import Required Libraries

Import all necessary libraries for our application.

**What each library does:**
- `gradio` - Creates the web interface
- `torch` - Handles GPU/CPU operations
- `transformers` - Loads AI models (Whisper, BART, T5)
- `langchain` - Builds the RAG Q&A system
- `FAISS` - Stores and searches vector embeddings
- `re` - Text pattern matching for parsing

In [ ]:
import gradio as gr
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import re
from typing import Optional, Tuple, List
from dataclasses import dataclass
import librosa  # For audio loading without ffmpeg
import numpy as np
import warnings

# Suppress all deprecation and future warnings
warnings.filterwarnings("ignore", message=".*forced_decoder_ids.*")
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# LangChain imports for RAG
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline  # NEW: Both from langchain_huggingface
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

print("✅ Libraries imported successfully!")
print(f"🔍 Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

## Step 3: Define Data Structure

Create a data class to store transcription results and metadata.

**TranscriptionResult stores:**
- `text` - The full transcription
- `summary` - Generated summary
- `key_points` - List of extracted key points
- `timestamps` - Timing information from Whisper

In [ ]:
@dataclass
class TranscriptionResult:
    """Store transcription results and metadata"""
    text: str
    summary: str = ""
    key_points: List[str] = None
    timestamps: dict = None

print("✅ Data structure defined!")

## Step 4: Create the VoiceScribe AI Class (Part 1 - Initialization)

This is the main class that handles all AI functionality. We'll define it in parts:

**Part 1 - `__init__` method:**
- Detects GPU/CPU
- Loads Whisper model for transcription
- Loads BART model for summarization
- Loads embeddings model for RAG
- Loads FLAN-T5 model for Q&A

**Note:** This will download ~7.5GB of models on first run and take 2-5 minutes to load.

In [ ]:
# Start of VoiceScribeAI class - we'll build it piece by piece

class VoiceScribeAI:
    """Main class for VoiceScribe AI functionality"""

    def __init__(self):
        """Initialize models and components"""
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.dtype = torch.float16 if self.device == 'cuda' else torch.float32

        print(f"🚀 Initializing VoiceScribe AI on {self.device}...")
        print("⏳ This will take 2-5 minutes on first run...\n")

        # Initialize transcription pipeline
        print("📥 [1/4] Loading Whisper model...")
        self.transcription_pipe = pipeline(
            "automatic-speech-recognition",
            model="openai/whisper-medium.en",
            device=0 if self.device == 'cuda' else -1,  # 0 for GPU, -1 for CPU
            return_timestamps=True
        )
        print("   ✅ Whisper loaded!\n")

        # Initialize summarization model
        print("📥 [2/4] Loading BART summarization model...")
        self.summarization_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
        self.summarization_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")
        if self.device == 'cuda':
            self.summarization_model = self.summarization_model.to(self.device)
        print("   ✅ BART loaded!\n")

        # Initialize embeddings for RAG
        print("📥 [3/4] Loading embeddings model...")
        self.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2"
        )
        print("   ✅ Embeddings loaded!\n")

        # Initialize QA model
        print("📥 [4/4] Loading Q&A model...")
        qa_model = "google/flan-t5-base"
        self.qa_pipeline = pipeline(
            "text2text-generation",
            model=qa_model,
            device=0 if self.device == 'cuda' else -1,
            max_length=512
        )
        self.llm = HuggingFacePipeline(pipeline=self.qa_pipeline)
        print("   ✅ Q&A model loaded!\n")

        # Storage for current session
        self.current_transcription: Optional[TranscriptionResult] = None
        self.vector_store: Optional[FAISS] = None
        self.qa_chain: Optional[RetrievalQA] = None

        print("="*60)
        print("✅ All models loaded successfully!")
        print("="*60)

print("✅ VoiceScribeAI class initialization defined!")

## Step 5: Add Transcription Method to VoiceScribeAI Class

Now we'll add the main transcription method that:
1. Takes an audio file as input
2. Transcribes it using Whisper
3. Generates a summary
4. Extracts key points
5. Sets up RAG for Q&A
6. Returns all results

**Note:** We're extending the class defined above, so make sure you ran the previous cell first.

In [ ]:
# Add transcription method to the class
def transcribe_audio_method(self, audio_file_path: str, progress=gr.Progress()) -> Tuple[str, str, str, str]:
    """
    Transcribe audio file and return results
    Returns: (transcription, summary, key_points, status_message)
    """
    if not audio_file_path:
        return "", "", "", "❌ Please upload an audio file first."

    try:
        progress(0, desc="Starting transcription...")

        # Load audio using librosa (no ffmpeg needed!)
        progress(0.1, desc="Loading audio file...")
        audio_array, sampling_rate = librosa.load(audio_file_path, sr=16000)
        
        # Transcribe audio
        progress(0.2, desc="Transcribing audio (this may take a few minutes)...")
        result = self.transcription_pipe({
            "array": audio_array,
            "sampling_rate": sampling_rate
        })
        transcription = result["text"]
        timestamps = result.get("chunks", [])

        # Store transcription
        self.current_transcription = TranscriptionResult(
            text=transcription,
            timestamps=timestamps
        )

        progress(0.5, desc="Generating summary...")
        summary = self.generate_summary(transcription)
        self.current_transcription.summary = summary

        progress(0.8, desc="Extracting key points...")
        key_points = self.extract_key_points(transcription)
        self.current_transcription.key_points = key_points
        key_points_text = "\n".join([f"• {point}" for point in key_points])

        # Set up RAG for Q&A
        progress(0.9, desc="Setting up Q&A system...")
        self.setup_rag(transcription)

        progress(1.0, desc="Complete!")

        status = f"✅ Transcription complete! ({len(transcription.split())} words)"
        return transcription, summary, key_points_text, status

    except Exception as e:
        error_msg = f"❌ Error during transcription: {str(e)}"
        print(error_msg)
        return "", "", "", error_msg

# Attach method to class
VoiceScribeAI.transcribe_audio = transcribe_audio_method
print("✅ Transcription method added!")

## Step 6: Add Remaining Core Methods

Now we'll add all the remaining methods to the VoiceScribeAI class:

**Methods being added:**
- `generate_summary()` - Uses BART to create summaries
- `extract_key_points()` - Extracts main ideas
- `setup_rag()` - Sets up vector database for Q&A
- `answer_question()` - Answers questions using RAG
- `search_transcript()` - Searches for keywords
- `_split_text()` - Helper method for text chunking

In [ ]:
# Add all remaining methods to VoiceScribeAI class

def generate_summary_method(self, text: str, max_length: int = 200) -> str:
    """Generate a concise summary of the text"""
    try:
        max_input_length = 1024
        chunks = self._split_text(text, max_input_length)
        summaries = []
        
        for chunk in chunks:
            inputs = self.summarization_tokenizer(
                chunk, max_length=max_input_length, truncation=True, return_tensors="pt"
            )
            if self.device == 'cuda':
                inputs = {k: v.to(self.device) for k, v in inputs.items()}
            
            summary_ids = self.summarization_model.generate(
                inputs["input_ids"], max_length=max_length, min_length=50,
                num_beams=4, length_penalty=2.0, early_stopping=True
            )
            summary = self.summarization_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
            summaries.append(summary)
        
        if len(summaries) > 1:
            combined = " ".join(summaries)
            inputs = self.summarization_tokenizer(
                combined, max_length=max_input_length, truncation=True, return_tensors="pt"
            )
            if self.device == 'cuda':
                inputs = {k: v.to(self.device) for k, v in inputs.items()}
            summary_ids = self.summarization_model.generate(
                inputs["input_ids"], max_length=max_length, min_length=50, num_beams=4
            )
            return self.summarization_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summaries[0]
    except Exception as e:
        return f"Error generating summary: {str(e)}"

def extract_key_points_method(self, text: str, num_points: int = 5) -> List[str]:
    """Extract key points from the text"""
    try:
        prompt = f"""Extract {num_points} key points or main ideas from the following text.
        List them as numbered points.

        Text: {text[:2000]}

        Key points:"""
        
        result = self.qa_pipeline(prompt, max_length=300, num_return_sequences=1)
        points_text = result[0]['generated_text']
        
        points = []
        lines = points_text.split('\\n')
        for line in lines:
            line = line.strip()
            line = re.sub(r'^\\d+[\\.\\)]\\s*', '', line)
            if line and len(line) > 10:
                points.append(line)
        
        if not points:
            sentences = text.split('.')
            points = [s.strip() + '.' for s in sentences[:num_points] if len(s.strip()) > 20]
        
        return points[:num_points]
    except Exception as e:
        sentences = text.split('.')
        return [s.strip() + '.' for s in sentences[:num_points] if len(s.strip()) > 20]

def setup_rag_method(self, text: str):
    """Set up RAG system for Q&A"""
    try:
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        chunks = text_splitter.split_text(text)
        self.vector_store = FAISS.from_texts(chunks, self.embeddings)
        
        qa_prompt_template = """Use the following pieces of context to answer the question at the end.
        If you don't know the answer, just say that you don't know, don't try to make up an answer.

        Context: {context}

        Question: {question}

        Answer:"""
        
        QA_PROMPT = PromptTemplate(template=qa_prompt_template, input_variables=["context", "question"])
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm, chain_type="stuff",
            retriever=self.vector_store.as_retriever(search_kwargs={"k": 3}),
            chain_type_kwargs={"prompt": QA_PROMPT}
        )
    except Exception as e:
        print(f"Error setting up RAG: {e}")
        self.qa_chain = None

def answer_question_method(self, question: str) -> str:
    """Answer questions about the transcription using RAG"""
    if not self.current_transcription:
        return "⚠️ Please transcribe an audio file first before asking questions."
    if not question or question.strip() == "":
        return "⚠️ Please enter a question."
    if not self.qa_chain:
        return "⚠️ Q&A system not ready. Please try transcribing the audio again."
    try:
        result = self.qa_chain.invoke({"query": question})
        return result.get('result', 'No answer found.')
    except Exception as e:
        return f"❌ Error answering question: {str(e)}"

def search_transcript_method(self, keyword: str) -> str:
    """Search for keyword in transcript"""
    if not self.current_transcription:
        return "⚠️ Please transcribe an audio file first."
    if not keyword or keyword.strip() == "":
        return "⚠️ Please enter a keyword to search."
    try:
        text = self.current_transcription.text
        keyword = keyword.strip().lower()
        sentences = text.split('.')
        matches = []
        
        for i, sentence in enumerate(sentences):
            if keyword in sentence.lower():
                context = sentence.strip()
                matches.append(f"[Match {len(matches) + 1}] ...{context}...")
        
        if matches:
            result = f"Found {len(matches)} occurrence(s) of '{keyword}':\\n\\n"
            result += "\\n\\n".join(matches)
            return result
        else:
            return f"No occurrences of '{keyword}' found in the transcript."
    except Exception as e:
        return f"❌ Error searching transcript: {str(e)}"

def split_text_method(self, text: str, max_length: int) -> List[str]:
    """Split text into chunks of max_length words"""
    words = text.split()
    chunks = []
    current_chunk = []
    current_length = 0
    
    for word in words:
        current_chunk.append(word)
        current_length += 1
        if current_length >= max_length:
            chunks.append(' '.join(current_chunk))
            current_chunk = []
            current_length = 0
    
    if current_chunk:
        chunks.append(' '.join(current_chunk))
    return chunks

# Attach all methods to class
VoiceScribeAI.generate_summary = generate_summary_method
VoiceScribeAI.extract_key_points = extract_key_points_method
VoiceScribeAI.setup_rag = setup_rag_method
VoiceScribeAI.answer_question = answer_question_method
VoiceScribeAI.search_transcript = search_transcript_method
VoiceScribeAI._split_text = split_text_method

print("✅ All methods added to VoiceScribeAI class!")
print("✅ VoiceScribeAI class is now complete!")

## Step 7: Create Gradio Interface

Build a beautiful web interface with multiple tabs:

**Tabs included:**
1. **Transcribe & Analyze** - Upload audio and get transcription, summary, and key points
2. **Chat with Audio** - Ask questions about the transcription using RAG
3. **Search Transcript** - Find keywords in the transcription
4. **About** - Information about the application

This function will create the complete UI and connect all the AI functionality.

In [ ]:
def create_gradio_interface():
    """Create and configure the Gradio interface"""
    
    # Initialize VoiceScribe AI (this loads all models)
    vs_ai = VoiceScribeAI()
    
    # Custom CSS for styling
    custom_css = """
    .gradio-container {
        font-family: 'Arial', sans-serif;
    }
    .header {
        text-align: center;
        padding: 20px;
        background: linear-gradient(135deg, #1a1a2e 0%, #16213e 50%, #0f3460 100%);
        color: white;
        border-radius: 10px;
        margin-bottom: 20px;
        box-shadow: 0 4px 15px rgba(0, 0, 0, 0.3);
    }
    """
    
    # Create interface
    with gr.Blocks(css=custom_css, title="VoiceScribe AI") as demo:
        gr.Markdown("""
        # 🎤 VoiceScribe AI
        ### Intelligent Audio Analysis Tool
        Transform your audio recordings into actionable insights with AI-powered transcription, summarization, and Q&A.
        """, elem_classes="header")
        
        with gr.Tab("📝 Transcribe & Analyze"):
            gr.Markdown("""
            ### Upload your audio file to get started
            Supports MP3, WAV, M4A, and other common audio formats.
            """)
            
            audio_input = gr.Audio(label="Upload Audio File", type="filepath")
            transcribe_btn = gr.Button("🚀 Transcribe & Analyze", variant="primary", size="lg")
            status_output = gr.Textbox(label="Status", interactive=False)
            
            with gr.Row():
                with gr.Column():
                    gr.Markdown("### 📄 Full Transcript")
                    transcript_output = gr.Textbox(
                        label="Transcription", lines=10,
                        placeholder="Transcript will appear here...", interactive=False
                    )
                
                with gr.Column():
                    gr.Markdown("### 📊 Summary")
                    summary_output = gr.Textbox(
                        label="Summary", lines=5,
                        placeholder="Summary will appear here...", interactive=False
                    )
                    
                    gr.Markdown("### 🎯 Key Points")
                    keypoints_output = gr.Textbox(
                        label="Key Points", lines=5,
                        placeholder="Key points will appear here...", interactive=False
                    )
        
        # Combined Chat & Search Tab
        with gr.Tab("💬 Chat & Search"):
            gr.Markdown("""
            ### Interact with your transcription
            Ask questions or search for specific keywords in your transcript.
            """)
            
            with gr.Row():
                # Left Column - Chat with Audio (Q&A)
                with gr.Column():
                    gr.Markdown("### 💬 Chat with Audio (Q&A)")
                    gr.Markdown("Ask natural language questions about your transcription.")
                    
                    question_input = gr.Textbox(
                        label="Ask a Question",
                        placeholder="e.g., What were the main topics discussed?",
                        lines=2
                    )
                    ask_btn = gr.Button("🤔 Get Answer", variant="primary")
                    answer_output = gr.Textbox(
                        label="Answer", lines=6,
                        placeholder="Answer will appear here...", interactive=False
                    )
                    
                    gr.Markdown("""
                    **Example Questions:**
                    - What did they say about training data?
                    - What are the action items mentioned?
                    - What problems were discussed?
                    """)
                
                # Right Column - Search Transcript
                with gr.Column():
                    gr.Markdown("### 🔍 Search Transcript")
                    gr.Markdown("Find all occurrences of a keyword or phrase.")
                    
                    search_input = gr.Textbox(
                        label="Enter Keyword",
                        placeholder="e.g., deadline, budget, meeting",
                        lines=2
                    )
                    search_btn = gr.Button("🔎 Search", variant="primary")
                    search_output = gr.Textbox(
                        label="Search Results", lines=6,
                        placeholder="Search results will appear here...", interactive=False
                    )
                    
                    gr.Markdown("""
                    **Search Tips:**
                    - Use single keywords for best results
                    - Try different variations (e.g., "meet", "meeting")
                    - Search is case-insensitive
                    """)
        
        with gr.Tab("ℹ️ About"):
            gr.Markdown("""
            ## About VoiceScribe AI
            
            VoiceScribe AI is an intelligent audio analysis tool that transforms your audio recordings into actionable insights.
            
            ### 🔹 Key Features:
            
            - **🎤 Automatic Transcription**: Converts audio to text using OpenAI's Whisper model
            - **🧠 Smart Summarization**: Generates clear summaries using BART
            - **📌 Key Points Extraction**: Lists the main topics and action items
            - **🔍 Searchable Transcript**: Quickly find parts of the recording by keyword
            - **💬 Chat with Audio (Q&A Mode)**: Ask natural questions using RAG
            
            ### 🛠️ Technology Stack:
            
            - **Transcription**: OpenAI Whisper (medium.en)
            - **Summarization**: Facebook BART-large-CNN
            - **Q&A**: Google FLAN-T5 with LangChain RAG
            - **Embeddings**: Sentence Transformers (all-MiniLM-L6-v2)
            - **Vector Store**: FAISS
            - **UI**: Gradio
            
            ### 📝 Usage Tips:
            
            1. Upload your audio file in the "Transcribe & Analyze" tab
            2. ⏳ **Wait for transcription to complete (may take a few minutes on CPU)**
            3. View your transcript, summary, and key points
            4. Go to "Chat & Search" tab to ask questions or search keywords
            
            ### 🚀 Powered by Open Source Models
            """)
        
        # Connect functions to buttons
        transcribe_btn.click(
            fn=vs_ai.transcribe_audio,
            inputs=[audio_input],
            outputs=[transcript_output, summary_output, keypoints_output, status_output]
        )
        
        ask_btn.click(
            fn=vs_ai.answer_question,
            inputs=[question_input],
            outputs=[answer_output]
        )
        
        search_btn.click(
            fn=vs_ai.search_transcript,
            inputs=[search_input],
            outputs=[search_output]
        )
    
    return demo

print("✅ Gradio interface function created!")

## Step 8: Launch the Application! 🚀

Now run this cell to launch VoiceScribe AI!

**What will happen:**
1. All AI models will be loaded (takes 2-5 minutes on first run)
2. A web interface will open
3. You'll get a local URL (e.g., http://127.0.0.1:7860)
4. You'll also get a public URL for sharing (e.g., https://xxxxx.gradio.live)

**After launching:**
- Click on the local URL to open the interface
- Upload an audio file in the "Transcribe & Analyze" tab
- Wait for processing
- Try asking questions and searching!

**Note:** The cell will keep running until you stop it (use the stop button in Jupyter).

In [ ]:
# Launch VoiceScribe AI!
print("🚀 Starting VoiceScribe AI...")
print("="*60)

# Create the interface (this will load all models)
demo = create_gradio_interface()

print("="*60)
print("✅ VoiceScribe AI is ready!")
print("🌐 Launching interface...")
print("="*60)

# Launch with public link
demo.launch(
    share=True,           # Create public shareable link
    debug=True,           # Show debug info
    show_error=True       # Show detailed errors
)

---

## 📖 How to Use the Application

Once the application is running, follow these steps:

### 1️⃣ Transcribe Audio

1. Go to the **"Transcribe & Analyze"** tab
2. Click to upload an audio file (or drag and drop)
3. Click **"🚀 Transcribe & Analyze"**
4. Wait 2-5 minutes (depending on file length)
5. View results: Transcript, Summary, and Key Points

### 2️⃣ Ask Questions (Q&A)

1. After transcribing, go to **"Chat with Audio"** tab
2. Type your question (e.g., "What were the main topics?")
3. Click **"🤔 Get Answer"**
4. View AI-generated answer based on the transcript

### 3️⃣ Search Transcript

1. Go to **"Search Transcript"** tab
2. Enter a keyword (e.g., "deadline")
3. Click **"🔎 Search"**
4. View all occurrences with context

---

## ⚙️ Customization Options

You can customize the application by modifying the code in the previous cells:

### Change Whisper Model (Step 4):
```python
# In the __init__ method, replace:
model="openai/whisper-medium.en"

# With one of these:
model="openai/whisper-tiny.en"    # Fastest (1GB)
model="openai/whisper-small.en"   # Balanced (2.5GB)  
model="openai/whisper-large"      # Best quality (10GB)
```

### Adjust Summary Length (Step 6):
```python
# In generate_summary_method, change:
max_length=200  # Make this larger for longer summaries
min_length=50   # Make this larger for minimum length
```

### Change Number of Key Points (Step 6):
```python
# In extract_key_points_method, change:
num_points=5  # Change to 3, 7, 10, etc.
```

### Modify RAG Chunk Size (Step 6):
```python
# In setup_rag_method, change:
chunk_size=500      # Larger = more context per chunk
chunk_overlap=50    # Overlap between chunks
```

---

## 🎯 Use Cases

VoiceScribe AI is perfect for:

- **📅 Meeting Minutes**: Automatically transcribe and summarize team meetings
- **🎙️ Podcast Analysis**: Extract key insights from podcast episodes
- **👥 Interview Transcription**: Convert interviews to searchable text
- **📚 Lecture Notes**: Transcribe educational content and generate study notes
- **🔬 Research Interviews**: Analyze qualitative research data
- **✍️ Content Creation**: Extract quotes and key points for articles
- **📞 Call Recordings**: Document important phone conversations
- **🎬 Video Content**: Transcribe YouTube videos or webinars

---

**Built with ❤️ using Open Source AI Models by Rohit**

**Happy Transcribing! 🎤**